<a href="https://colab.research.google.com/github/marino1846067/python/blob/main/3_MoreFuncs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FUNCTIONS: ADVANCED TOPICS
This notebook covers some of the useful features of Python functions

## 1. Returning multiple values
In C++ a function can return only one value
```c++
T function(args) {
 T val;
 // calculations
 return val;
}
```
where `T` can be any type or class.

**Python functions can return an arbitrary number of values (of arbitrary type combinations).**

In [ ]:
def xplus(x):
    return x+1, x+2, x+3, x+4

print(type(xplus(3)))

output = xplus(4)
print(type(output))
print(output)

<class 'tuple'>
<class 'tuple'>
(5, 6, 7, 8)


In [ ]:
list_output = list(xplus(2.2))
print(list_output)

[3.2, 4.2, 5.2, 6.2]


### Example: calculating boost parameters

Let's compute simple kinematic information and boost parameters.

For simplcity we assume the momentum along the *x* axis, but you should **TRY TO GENERALIZE THIS EXAMPLE.**

In [ ]:
import math as m

m_pi = 0.140 # GeV
p_pi = 1.2   # GeV

def make_p4(mass, p):
    return [m.sqrt(mass**2 + p**2), p, 0, 0] # momentum along the x axis as a list [E, px, py, pz]

p4_pi = make_p4(m_pi, p_pi)
print("pi 4-momentum:", p4_pi)

def boost_params(p4):
    p = m.sqrt(p4[1]**2 + p4[2]**2 + p4[3]**2)
    E = p4[0]
    mass = m.sqrt(E**2 - p**2)
    return p/E, E/mass, p/mass # beta, gamma, beta * gamma (as a tuple)

beta_pi, gamma_pi, betagamma_pi = boost_params(p4_pi)

print("pi boost: beta = {0:.3f}, gamma = {1:.3f}, beta*gamma = {2:.3f}".format(beta_pi, gamma_pi, betagamma_pi))

pi 4-momentum: [1.2081390648431165, 1.2, 0, 0]
pi boost: beta = 0.993, gamma = 8.630, beta*gamma = 8.571


## 2. The `_` variable

If you return more values you need to make sure that all of them are used when calling the function.

#### Example: suppose we only need $\beta$ and $\gamma$ and not $\beta\gamma$.

In [ ]:
m_B = 5.279 # GeV
p_B = 0.3   # GeV

beta_B, gamma_B = boost_params(make_p4(m_B, p_B))
print(beta_B)

ValueError: ignored

You are forced to have 3 variables to write to, in order for the function to work.

This can be tedious because at times you might not need all these returned values, or simply do not care. Fear not: Python has a solution for this as well.

The `_`  is a special variable that can be used for a number of purposes. One of them is to ignore values we do not care about.

Suppose we want to use only $\beta_B$.

In [ ]:
beta_B, *_ = boost_params(make_p4(m_B, p_B))

print("B beta:", beta_B)

B beta: 0.05673740118652557


In [ ]:
print(_)
print(type(_))
print(len(_))

[1.0016134628566604, 0.05682894487592347]
<class 'list'>
2


In this case `*_` means that 0 or more vales are unpacked and assigned to `_`.

Here `_` is a list of 2 objects. You can use `_` like any other variable.

In [ ]:
print(_)
print(_[0], _[1])

[1.0016134628566604, 0.05682894487592347]
1.0016134628566604 0.05682894487592347


#### Similarly

In [ ]:
a, b, *_ = xplus(13)
print(a, b, _)

a, b, c, _ = xplus(13)
print(a, b, c, _)

a, b, c, *_ = xplus(13)
print(a, b, c, _)

a, _, c, _ = xplus(13)
print(a, c, _)

a, *_, d = xplus(13)
print(a, d, _)

14 15 [16, 17]
14 15 16 17
14 15 16 [17]
14 16 17
14 17 [15, 16]


#### Watch out, however: only one `*_` is allowed

In [ ]:
*_, c, *_ = xplus(13)

SyntaxError: ignored

#### If the multiple values have special meanings, using a dictionary should be considered

In [ ]:
def boost_dict(p4):
    p = m.sqrt(p4[1]**2 + p4[2]**2 + p4[3]**2)
    E = p4[0]
    mass = m.sqrt(E**2 - p**2)
    return {'beta': p/E, 'gamma': E/mass, 'betagamma': p/mass}

m_mu = 0.106 # GeV
p_mu = 0.020 # GeV

boost_mu = boost_dict(make_p4(m_mu, p_mu))

print(boost_mu)

print("mu beta: ", boost_mu['beta'])

{'beta': 0.1854078591978247, 'gamma': 1.0176442686914566, 'betagamma': 0.18867924528301888}
mu beta:  0.1854078591978247


## 3. Anonymous (`lambda`) functions 
`lambda` functions are a special class of functions that consist of a simple single statement.

Suppose we want to compute `1 + x**2 - x**3` for elements of a list, using a comprehension.

In [ ]:
import random as r

def myfunc(x):
    return 1 + x**2 - x**3

def apply_to_list(alist, f):
    return [f(x) for x in alist]

alist = [r.normalvariate(1., 0.3) for i in range(5)]

print(alist)
print(apply_to_list(alist, myfunc))

[1.010836448599934, 0.8067165649899771, 0.9336191540146304, 1.0045731292260436, 0.8904494912048868]
[0.9889274216543622, 1.1257872390605097, 1.0578605142271245, 0.9953849481119323, 1.0868626308929965]


Function `myfunc()` has really no other use other than when applied to a list. So its name is basically useless.

Further, if we wanted now to apply a new function we would need to define a new useless function.

Rather than definining a standard function with 
```python
def myfunc(x):
    return 1 + x**2 - x**3
```
we can do something more light weight.

We can create functions on the fly which do not have a name. Technically it means the function object does not have the `__name__` attribute [remember: also functions are objects in Python, so they have attributes and `__name__` is one of them].

The solution with a `lambda` function is quite simple.

In [ ]:
print(apply_to_list(alist, myfunc))
print(apply_to_list(alist, lambda x: 1 + x**2 - x**3))

[0.9889274216543622, 1.1257872390605097, 1.0578605142271245, 0.9953849481119323, 1.0868626308929965]
[0.9889274216543622, 1.1257872390605097, 1.0578605142271245, 0.9953849481119323, 1.0868626308929965]


### Sorting lists with lambda functions
A typical use of `lambda` functions is list sorting.

In [ ]:
vals = [r.uniform(0., 3.) for i in range(5)]
print("Original data:", vals)
print("Formated data:", ["{0:0.3f}".format(x) for x in vals])

vals.sort()
print("Sorted data:", ["%.3f"%x for x in vals])

vals.sort(key=lambda x: m.sin(x))
print("Data sorted by sin with lambda:",["%.3f"%x for x in vals])

vals.sort(key=m.sin)
print("Data sorted by sin without lambda:",["%.3f"%x for x in vals])

Original data: [0.1419598536959179, 1.8554390384353914, 1.922361201436153, 0.6526796067279187, 2.940977870549786]
Formated data: ['0.142', '1.855', '1.922', '0.653', '2.941']
Sorted data: ['0.142', '0.653', '1.855', '1.922', '2.941']
Data sorted by sin with lambda: ['0.142', '2.941', '0.653', '1.922', '1.855']
Data sorted by sin without lambda: ['0.142', '2.941', '0.653', '1.922', '1.855']


As an additonal use, we can sort the numbers based on unique numerals appearing in the number itself

In [ ]:
print(["{0:0.3f}".format(x) for x in vals])

new_vals = [set("%.3f"%x) for x in vals]
print(new_vals)

vals.sort(key=lambda x: len(set("%.3f"%x)))
print(["%.3f"%x for x in vals])

['1.922', '1.855', '0.142', '2.941', '0.653']
[{'.', '1', '2', '9'}, {'8', '.', '1', '5'}, {'1', '0', '4', '2', '.'}, {'.', '1', '4', '2', '9'}, {'6', '0', '3', '5', '.'}]
['1.922', '1.855', '0.142', '2.941', '0.653']


## 4. Functions with arbitrary number of arguments

As seen for example with the `print()` function, functions can have a variable number of arguments. The same behaviour can easily be defined for any custom defined function for both **positional and keyword arguments**.

### Positional arguments

Additional arguments are taken via the special `*arg` argument which is a **tuple** of additional positional arguments.

In [ ]:
def myfunc(a, *arg):
    print("Positional arguments: %s %s" % (a, arg))
    if len(arg):
        for x in arg:
            print('[%s]\t'%x)
        print(type(arg), '\n')

In [ ]:
myfunc(1.1)

Positional arguments: 1.1 ()


In [ ]:
myfunc('hello')

Positional arguments: hello ()


In [ ]:
myfunc()

TypeError: ignored

In [ ]:
myfunc(-0.2, 0.3, 'hello')

Positional arguments: -0.2 (0.3, 'hello')
[0.3]	
[hello]	
<class 'tuple'> 



In [ ]:
myfunc(-0.2, 0.3, 'hello', 'goodbye', -2, 100)

Positional arguments: -0.2 (0.3, 'hello', 'goodbye', -2, 100)
[0.3]	
[hello]	
[goodbye]	
[-2]	
[100]	
<class 'tuple'> 



### Keyword arguments

For optional keyword arguments, the `**karg` feature is used.  This is a **dictionary**.

In [ ]:
def myfunc2(a, mu=0.0, sig=0.1, **karg):
    print("a: %s"%(a))
    print("Keyword arguments: %s %s %s" % (mu, sig, karg))
    if len(karg):
        for x in karg:
            print('[%s]\t' % x)
        print(type(karg), '\n')

In [ ]:
myfunc2(0.11111)

a: 0.11111
Keyword arguments: 0.0 0.1 {}


In [ ]:
myfunc2(0.3, sig=0.5)

a: 0.3
Keyword arguments: 0.0 0.5 {}


In [ ]:
myfunc2(0.3, color='red')

a: 0.3
Keyword arguments: 0.0 0.1 {'color': 'red'}
[color]	
<class 'dict'> 



In [ ]:
myfunc2(3, color='red', mu=0.6)

a: 3
Keyword arguments: 0.6 0.1 {'color': 'red'}
[color]	
<class 'dict'> 



The additional keyword arguments are stored as a dictionary.

In [ ]:
def myfunc3(a,mu=0.0, sig=0.1, **karg):
    print("a: %s"%(a))
    print("Keyword arguments: %s %s %s" % (mu, sig, karg))
    if len(karg):
        for x in karg.keys():
            print('[%s = %s]\t'%(x, karg[x]))
        print(type(karg), '\n')
        
myfunc3(0.1)

a: 0.1
Keyword arguments: 0.0 0.1 {}


In [ ]:
myfunc3(0.3, color='red', mu=0.6)

a: 0.3
Keyword arguments: 0.6 0.1 {'color': 'red'}
[color = red]	
<class 'dict'> 



### Combine both positional and keyword arguments for the most generic function

In [ ]:
def myfunc4(a, *arg, mu=0, sig=1, **karg):
    print("Myfunc4 called")
    print("Positional:  a: %s.     Optional: %s" % (a, arg))
    if len(arg):
        for x in arg:
            print('[%s]\t' % x)
        print(type(arg), '\n')
    print("keyword: %s %s %s" % (mu, sig, karg))    
    if len(karg):
        for x in karg.keys():
            print('[%s = %s]\t' % (x, karg[x]))
        print(type(karg), '\n')
    
myfunc4(-0.1)

Myfunc4 called
Positional:  a: -0.1.     Optional: ()
keyword: 0 1 {}


In [ ]:
myfunc4(-0.1, 10.1)

Myfunc4 called
Positional:  a: -0.1.     Optional: (10.1,)
[10.1]	
<class 'tuple'> 

keyword: 0 1 {}


In [ ]:
myfunc4(-0.1, mu=10.1)

Myfunc4 called
Positional:  a: -0.1.     Optional: ()
keyword: 10.1 1 {}


In [ ]:
myfunc4(0.3, 'x', 'y', 0.9, color='red', mu=0.6, thick=1.1, fill='true')

Myfunc4 called
Positional:  a: 0.3.     Optional: ('x', 'y', 0.9)
[x]	
[y]	
[0.9]	
<class 'tuple'> 

keyword: 0.6 1 {'color': 'red', 'thick': 1.1, 'fill': 'true'}
[color = red]	
[thick = 1.1]	
[fill = true]	
<class 'dict'> 



# COMMAND LINE ARGUMENTS

Since we covered function arguments, it is only natural to wonder how to provide arguments to a Python script being run from command line.

The `sys` module grants easy access to command line arguments as a list. An example is in `examples/Python/cmd_line_args.py`, which reads:

```Python
import sys, os

print("Running " + __file__)

print("Running " + os.path.basename(__file__))

print("Program called with %d arguments" % len(sys.argv))

for a in sys.argv:
    print(a)
```

Run this in a terminal (with `python3`!) to see what happens.

# READY FOR `examples/Python/4-NumPy.ipynb`!